# RCE_Lab Result

In [1]:
from utils import *
from sklearn.metrics import confusion_matrix

In [2]:
manalyze_result = load_nested_json("manalyze_result_rcelab.jsonl", process_manalyze_result)
pypacker_result = load_nested_json("pypacker_result_rcelab.jsonl", process_pypacker)
pypeid_result   = load_nested_json("pypeid_result_rcelab.jsonl", process_peid)

In [3]:
df = pd.merge(pd.merge(pypacker_result, pypeid_result, on="name"), manalyze_result)

In [4]:
df.head()

,name,pypacker_suspicions,pypacker_detections,peid_packed,peid_PEiD,manalyze_result_level,manalyze_result_output
0,/home/nanoha/chome/Desktop/RCE_Lab_bins/!EP (E...,[],[Entry point 0x6b0ff in irregular section(s): ...,True,"[AHPack_V01_FEUERRADER, AHpack_01_FEUERRADER_h...",2.0,[Section .text is both writable and executable...
1,/home/nanoha/chome/Desktop/RCE_Lab_bins/!EP (E...,[],[Entry point 0xb3000 in irregular section(s): ...,True,[],2.0,"[Unusual section name found:, Section is both..."
2,/home/nanoha/chome/Desktop/RCE_Lab_bins/!EProt...,[],[Entry point 0x6b010 in irregular section(s): ...,False,[AHTeam_EP_Protector_03_fake_PCGuard_403_415_F...,2.0,[Unusual section name found: !eprot]
3,/home/nanoha/chome/Desktop/RCE_Lab_bins/.NET R...,[],[],True,[Microsoft_Visual_Cpp_v50v60_MFC],NaN,None
4,/home/nanoha/chome/Desktop/RCE_Lab_bins/123111...,[],[Entry point 0x692b0 in irregular section(s): ...,True,"[bambam_V004_bedrock, DragonArmor_Orient_addit...",2.0,"[Unusual section name found: Xiao, The PE only..."


In [5]:
def process_pypacker_detection_reason(msg):
    return [PyPackerDetectionReason.msg_to_enum(i) for i in msg if PyPackerDetectionReason.msg_to_enum(i)] if msg else []

def process_manalyze_detection_reason(msg):
    return [ManalyzeDetectionReason.msg_to_enum(i) for i in msg if ManalyzeDetectionReason.msg_to_enum(i)] if msg else []

df["pypacker_reason"] = df["pypacker_suspicions"].apply(lambda x: process_pypacker_detection_reason(x)) + df["pypacker_detections"].apply(lambda x: process_pypacker_detection_reason(x))
df["manalyze_reason"] = df["manalyze_result_output"].apply(lambda x: process_manalyze_detection_reason(x))

In [6]:
df["pypacker_packed"] = df["pypacker_reason"].apply(lambda x: len(x) != 0) | df["pypacker_reason"].apply(lambda x: len(x) != 0)
df["manalyze_packed"] = df["manalyze_reason"].apply(lambda x: len(x) != 0 if x else False)
df["pypacker_manalyze_packed"] = df["pypacker_packed"] | df["manalyze_packed"]
df["peid_packed_by_sign"] = df["peid_PEiD"].apply(lambda x: len(x) != 0)

In [7]:
def show_coverage(df, key):
    print(key, df[key].sum() / len(df) * 100.)

In [8]:
# comparison of detection coverage among PEiD, PyPacker, and Manalyze
show_coverage(df, "peid_packed")
show_coverage(df, "pypacker_packed")
show_coverage(df, "manalyze_packed")

peid_packed 82.6311263972485
pypacker_packed 80.91143594153053
manalyze_packed 79.96560619088564


In [9]:
# merged result
print("pypacker_manalyze_peid_packed", 100.0 * (df["peid_packed"] | df["pypacker_manalyze_packed"]).sum() / len(df)) # result of Manalyze | PEiD | PyPacker
print("pypacker_peid_packed", 100.0 * (df["peid_packed"] | df["pypacker_packed"]).sum() / len(df)) # result of PEiD | PyPacker
print("manalyze_peid_packed", 100.0 * (df["peid_packed"] | df["manalyze_packed"]).sum() / len(df)) # result of PEiD | Maalyze

pypacker_manalyze_peid_packed 95.87274290627687
pypacker_peid_packed 95.09888220120378
manalyze_peid_packed 95.18486672398969


In [10]:
df["peid_not_packed_pypacker_packed"] = ~df["peid_packed"] & df["pypacker_packed"] # detected as "not-packed" by PEiD, detected as "packed" by PyPacker
df["peid_not_packed_manalyze_packed"] = ~df["peid_packed"] & df["manalyze_packed"] # detected as "not-packed" by PEiD, detected as "packed" by Manalyze

In [11]:
from collections import defaultdict

def flatten_list(l):
    result = list()
    for i in l:
        result += i
    return result

def calc_hist(l):
    d = defaultdict(int)
    for i in l:
        d[i] += 1
    return d

In [12]:
# detection reason result (PyPacker)
# About 52% of files are detected as "packed" because they have a bad entry point address.
calc_hist(flatten_list(list(df[df["peid_not_packed_pypacker_packed"]]["pypacker_reason"])))

defaultdict(int,
            {<PyPackerDetectionReason.BAD_ENTRY_POINT: 4>: 151,
             <PyPackerDetectionReason.NONSTANDARD_SECTIONNAME: 2>: 77,
             <PyPackerDetectionReason.SECTION_NAME_IS_KNOWN: 3>: 65})

In [13]:
# detection reason result (Manalyze)
# Most of the files are detected as "packed" because they have an unusual section name.
calc_hist(flatten_list(list(df[df["peid_not_packed_manalyze_packed"]]["manalyze_reason"])))

defaultdict(int,
            {<ManalyzeDetectionReason.UNUSUAL_SECTION_NAME: 1>: 415,
             <ManalyzeDetectionReason.BROKEN_RITCH_HEADER: 5>: 4})